# Libraries

In [11]:
import os
from pathlib import Path
import numpy as np
import radiomics
from radiomics import featureextractor
import cv2 as cv
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from tqdm import tqdm

In [12]:
features_path = Path.cwd()
notebooks_path = features_path.parent
repo_path = notebooks_path.parent
os.chdir(str(features_path))
#print current working directory
print(os.getcwd())

/home/ricardino/Documents/MAIA/tercer_semestre/CAD/Projecte/Machine_Learning/notebooks/features


# Function

In [13]:
class path_label():
    """Class to access paths and labels from csv
    """
    def __init__(self, meta=pd.read_csv(str(repo_path) + '/data/meta_info.csv', sep='\t'), classif='binary', set_name='train') -> None:
        meta = meta.loc[meta['classif'] == classif] #Filter by classif
        meta = meta.loc[meta['set'] == set_name] #Filter by set
        self.paths = list(meta.path)
        self. labels = np.array(meta.label)
        self.FOV_x1 = np.array(meta.FOV_x1, dtype=np.int16)
        self.FOV_x2 = np.array(meta.FOV_x2, dtype=np.int16)
        self.FOV_y1 = np.array(meta.FOV_y1, dtype=np.int16)
        self.FOV_y2 = np.array(meta.FOV_y2, dtype=np.int16)
        
#create class to call patient and its information
class patient():
    """Class to access patient information
    """
    def __init__(self, info = path_label(), num=0) -> None:
        self.path = info.paths[num]
        self.label = info.labels[num]
        self.FOV_x1 = info.FOV_x1[num]
        self.FOV_x2 = info.FOV_x2[num]
        self.FOV_y1 = info.FOV_y1[num]
        self.FOV_y2 = info.FOV_y2[num]
        self.image = cv.imread(str(repo_path) +"/"+ self.path)
        self.image = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
        self.image = self.image[self.FOV_x1:self.FOV_x2, self.FOV_y1:self.FOV_y2]
        self.HSV = cv.imread(str(repo_path) +"/"+ self.path, cv.IMREAD_COLOR)
        self.HSV = cv.cvtColor(self.HSV, cv.COLOR_BGR2HSV)
        self.HSV = self.HSV[self.FOV_x1:self.FOV_x2, self.FOV_y1:self.FOV_y2]

def extractor_setting():

  settings = {}
  settings['binWidth'] = 1 #To ensure a 1 pixel width for the histogram once the image has been requantized to 8 gray levels.
  settings['correctMask'] = True #To ensure the mask and the image are in the same coordinates
  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  #Enable all first order features
  extractor.enableFeatureClassByName('glcm')
  extractor.enableFeatureClassByName('glszm')
  extractor.enableFeatureClassByName('glrlm')
  extractor.enableFeatureClassByName('ngtdm')
  extractor.enableFeatureClassByName('gldm')
  
  
  print('Extraction parameters:\n\t', extractor.settings)
  print('Enabled filters:\n\t', extractor.enabledImagetypes)
  print('Enabled features:\n\t', extractor.enabledFeatures)
  return extractor

In [14]:
def all_texture(pat, extractor):

    im_HSV =  pat.HSV[:,:,2]#Value channel image
    sitk_im = sitk.GetImageFromArray(im_HSV) #Pass to sitk objet
    #mask
    mask = np.ones(im_HSV.shape).astype(np.uint8) #Create maskwith same shape as image
    mask[-1,-1]=0 #Hack to allow full image segmentation. Last value is set to 0 so it wont be counted in the GLCM computation
    sitk_mask = sitk.GetImageFromArray(mask) #Pass to sitk object
    #Extract
    result = extractor.execute(sitk_im,sitk_mask) # Extract features
    features = np.array(list(result.values())[-75:]) #save as array 18 features
    
    return features

In [15]:
extractor = extractor_setting() #extractor settings 

classif='binary'; set_name='test'
meta = pd.read_csv(str(repo_path) + '/data/meta_test.csv', sep='\t')
info = path_label(meta, classif=classif, set_name=set_name)
length = len(info.paths) #number of images
fv_matrix = np.zeros(shape=(length,75),dtype=np.float64) #Empty numpy array to store feature vectors

for i in tqdm(range(length)):
    pat = patient(info=info, num=i)
    fv_matrix[i] = all_texture(pat, extractor)
    
ftype='texture'; f_name='All_texture'
with open(str(repo_path)+ f'/data/features/{ftype}/{classif}_{set_name}_{ftype}_{f_name}_fv.p', 'wb') as handle:
    pickle.dump(fv_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 1, 'correctMask': True}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': [], 'glszm': [], 'glrlm': [], 'ngtdm': [], 'gldm': []}


 84%|████████▍ | 851/1015 [05:56<01:05,  2.51it/s]GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
100%|██████████| 1015/1015 [07:02<00:00,  2.40it/s]
